In [1]:
### REQUIREMENTS
import numpy as np
import math
import pandas as pd

import sys
### we do a lot of requests during the scrapping. Some of them with requests package, some of them with urllib
import requests
from bs4 import BeautifulSoup

# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import getpass
import io

import datetime as dt
# for simple paralel computing:
from concurrent.futures import ThreadPoolExecutor

import geopandas as gpd

import sddk

In [2]:
### configure session and groupurl
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [3]:
# request geojson with geographies
resp = requests.get("https://edh-www.adw.uni-heidelberg.de/download/edhGeographicData.json", headers={'User-Agent': ''})
resp

<Response [200]>

In [22]:
resp.json()["features"][:10]

[{'geometry': {'coordinates': [23.451654999, 38.497692999], 'type': 'Point'},
  'properties': {'ancient_findspot': 'Anthedon',
   'uri': 'https://edh-www.adw.uni-heidelberg.de/edh/geographie/0',
   'pleiades_uri': 'https://pleiades.stoa.org/places/540639',
   'trismegistos_geo_uri': 'https://www.trismegistos.org/place/15743'},
  'type': 'Feature'},
 {'type': 'Feature',
  'properties': {'pleiades_uri': 'https://pleiades.stoa.org/places/570316',
   'trismegistos_geo_uri': 'https://www.trismegistos.org/place/15746',
   'ancient_findspot': 'Isthmia',
   'uri': 'https://edh-www.adw.uni-heidelberg.de/edh/geographie/3'},
  'geometry': {'type': 'Point', 'coordinates': [22.992664, 37.9158514]}},
 {'type': 'Feature',
  'properties': {'ancient_findspot': 'Lebadea',
   'uri': 'https://edh-www.adw.uni-heidelberg.de/edh/geographie/4',
   'trismegistos_geo_uri': 'https://www.trismegistos.org/place/1851'},
  'geometry': {'coordinates': [23.273345, 38.221257], 'type': 'Point'}},
 {'geometry': {'coordin

In [23]:
geo_uri_dict = {} # dictionary arranged by uris
for feature in resp.json()["features"]:
    try:
        geo_uri_dict[feature["properties"]["uri"].rpartition("/")[2]] = feature
    except:
        pass

In [24]:
geo_uri_dict

{'0': {'geometry': {'coordinates': [23.451654999, 38.497692999],
   'type': 'Point'},
  'properties': {'ancient_findspot': 'Anthedon',
   'uri': 'https://edh-www.adw.uni-heidelberg.de/edh/geographie/0',
   'pleiades_uri': 'https://pleiades.stoa.org/places/540639',
   'trismegistos_geo_uri': 'https://www.trismegistos.org/place/15743'},
  'type': 'Feature'},
 '3': {'type': 'Feature',
  'properties': {'pleiades_uri': 'https://pleiades.stoa.org/places/570316',
   'trismegistos_geo_uri': 'https://www.trismegistos.org/place/15746',
   'ancient_findspot': 'Isthmia',
   'uri': 'https://edh-www.adw.uni-heidelberg.de/edh/geographie/3'},
  'geometry': {'type': 'Point', 'coordinates': [22.992664, 37.9158514]}},
 '4': {'type': 'Feature',
  'properties': {'ancient_findspot': 'Lebadea',
   'uri': 'https://edh-www.adw.uni-heidelberg.de/edh/geographie/4',
   'trismegistos_geo_uri': 'https://www.trismegistos.org/place/1851'},
  'geometry': {'coordinates': [23.273345, 38.221257], 'type': 'Point'}},
 '5':

In [25]:
sddk.write_file("geo_uri_dict.json", geo_uri_dict, conf)

Your <class 'dict'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/geo_uri_dict.json"


In [13]:
# change geodataframe object to ordinary dataframe
geo_data_df = pd.DataFrame(edh_geo) 

In [12]:
#geo_data_df.set_index("uri", inplace=True)

KeyError: "None of ['uri'] are in the columns"

In [14]:
geo_data_df.head(5)

,ancient_findspot,uri,pleiades_uri,trismegistos_geo_uri,modern_findspot,geonames_uri,findspot_(village/street),geometry
0,Anthedon,https://edh-www.adw.uni-heidelberg.de/edh/geog...,https://pleiades.stoa.org/places/540639,https://www.trismegistos.org/place/15743,None,None,None,POINT (23.45165 38.49769)
1,Isthmia,https://edh-www.adw.uni-heidelberg.de/edh/geog...,https://pleiades.stoa.org/places/570316,https://www.trismegistos.org/place/15746,None,None,None,POINT (22.99266 37.91585)
2,Lebadea,https://edh-www.adw.uni-heidelberg.de/edh/geog...,None,https://www.trismegistos.org/place/1851,None,None,None,POINT (23.27334 38.22126)
3,Mantinea,https://edh-www.adw.uni-heidelberg.de/edh/geog...,None,https://www.trismegistos.org/place/1851,None,None,None,POINT (23.27334 38.22126)
4,Megalopolis,https://edh-www.adw.uni-heidelberg.de/edh/geog...,https://pleiades.stoa.org/places/321678,https://www.trismegistos.org/place/1327,None,None,None,POINT (22.87611 38.43582)


In [15]:
sddk.write_file("SDAM_data/EDH/EDH_geographies_raw_2020-09-15.json", geo_data_df, conf)

OverflowError: Maximum recursion level reached

In [64]:
conf[0].put(conf[1] + "SDAM_data/EDH/EDH_geographies_raw.json")

<Response [204]>

# Geographies on the province basis

In [5]:
### extract names of provinces
headers = {'User-Agent': ''}
response = requests.get("https://edh-www.adw.uni-heidelberg.de/data/api/terms/province", headers=headers)
response
json_data = response.json()
print(str(json_data)[:200])

{'provinces': {'Ach': 'Achaia', 'Aeg': 'Aegyptus', 'Aem': 'Aemilia (Regio VIII)', 'Afr': 'Africa Proconsularis', 'AlC': 'Alpes Cottiae', 'AlG': 'Alpes Graiae', 'AlM': 'Alpes Maritimae', 'AlP': 'Alpes 


In [6]:
json_data["provinces"].keys()

dict_keys(['Ach', 'Aeg', 'Aem', 'Afr', 'AlC', 'AlG', 'AlM', 'AlP', 'ApC', 'Aqu', 'Ara', 'Arm', 'Asi', 'Ass', 'Bae', 'Bar', 'Bel', 'BiP', 'BrL', 'Bri', 'Cap', 'Cil', 'Cor', 'Cre', 'Cyp', 'Cyr', 'Dac', 'Dal', 'Epi', 'Etr', 'Gal', 'GeI', 'GeS', 'HiC', 'Inc', 'Iud', 'LaC', 'Lig', 'Lug', 'Lus', 'LyP', 'MaC', 'MaT', 'Mak', 'Mes', 'MoI', 'MoS', 'Nar', 'Nor', 'Num', 'PaI', 'PaS', 'Pic', 'Rae', 'ReB', 'Rom', 'Sam', 'Sar', 'Sic', 'Syr', 'Thr', 'Tra', 'Tri', 'Umb', 'Val', 'VeH'])

In [25]:
resp = requests.get("https://edh-www.adw.uni-heidelberg.de/data/api/geography?limit=1", headers=headers)
total = resp.json()["total"]
total

30536

In [33]:
resp = requests.get("https://edh-www.adw.uni-heidelberg.de/data/api/geography?limit=" + str(10000), headers=headers)
items = resp.json()["items"]
len(items)

10000

In [ ]:
###  generate a list of provinces, resp. their abbreviations
%%time

json_data = requests.get("https://edh-www.adw.uni-heidelberg.de/data/api/terms/province").json()
provinces = json_data["provinces"].keys()

### make the basic URL form
URL_form_geo = "https://edh-www.adw.uni-heidelberg.de/data/api/geography/search?"

### parse all find spot for each province 
### at them to the list
geo_data = []
for province in provinces:
  total = requests.get(URL_form_geo + "province=" + province).json()["total"]
  response = requests.get(URL_form_geo + "province=" + province + "&limit=" + str(total))
  geo_data.extend(response.json()["items"])
  print(province, total)

Ach 183
Aeg 65
Aem 162
Afr 1400
AlC 59
AlG 31
AlM 173
AlP 79
ApC 204
Aqu 212
Ara 103
Arm 2
Asi 182
Ass 0
Bae 1305
Bar 89
Bel 429
BiP 105
BrL 79
Bri 1950
Cap 31
Cil 48
Cor 15
Cre 29
Cyp 21
Cyr 24
Dac 996
Dal 2194
Epi 77
Etr 356
Gal 89
GeI 1064
GeS 2661
HiC 2553
Inc 7
Iud 83
LaC 654
Lig 131
Lug 254
Lus 746
LyP 37
MaC 298
MaT 43
Mak 434
Mes 9
MoI 888
MoS 790
Nar 635
Nor 1504
Num 333
PaI 1316
PaS 1623
Pic 85
Rae 729
ReB 22
Rom 734
Sam 313
Sar 81
Sic 47
Syr 154
Thr 257
Tra 111
Tri 0
Umb 212
Val 0
VeH 479
CPU times: user 1.75 s, sys: 143 ms, total: 1.89 s
Wall time: 2min 19s


In [ ]:
### total amount of findspots
len(geo_data)

29979

In [ ]:
geo_data_df = pd.DataFrame(geo_data)
geo_data_df.set_index("uri", inplace=True)
geo_data_df.head(5)

,country,province,last_update,id,find_spot_ancient,coordinates,region,find_spot_modern,find_spot,pleiades_uri,geonames_uri
uri,,,,,,,,,,,
https://edh-www.adw.uni-heidelberg.de/edh/geographie/10,Greece,Achaia,2011-07-14,10,Ptoion,"37.05,25.19",NaN,NaN,NaN,NaN,NaN
https://edh-www.adw.uni-heidelberg.de/edh/geographie/100,Greece,Achaia,2012-03-20,100,Athenae,"37.983175,23.716647",Attikí,Athínai,Kerameikos,https://pleiades.stoa.org/places/579885,NaN
https://edh-www.adw.uni-heidelberg.de/edh/geographie/101,Greece,Achaia,2012-03-20,101,Athenae,"37.983175,23.716647",Attikí,Athínai,Kerameikos bei Kirche Haghia Triada,https://pleiades.stoa.org/places/579885,NaN
https://edh-www.adw.uni-heidelberg.de/edh/geographie/102,Greece,Achaia,2012-03-20,102,Colonia Augusta Aroe Patrae,"38.251123,21.741943",Dytikí Elláda,Pátrai,"Kirche, sekundär verwendet",NaN,NaN
https://edh-www.adw.uni-heidelberg.de/edh/geographie/103,Greece,Achaia,2012-03-20,103,Colonia Laus Iulia Corinthus,"37.939865,22.928467",Pelopónissos,Kórinthos,Kranion,NaN,NaN


## Upload the data to sciencedata.dk

In [ ]:
### put your dataframe data into this folder
sddl.write_file("SDAM_data/EDH/EDH_geographies_raw.json", "df", conf)

<Response [201]>